In [1]:
#THIS FUNCTION IS NOT WORKING!!!

def registration (fixed_image, moving_image):
    """ ATTENTION: NOT WORKING!
    This function do the registration of a moving image over a fixed image.
    
    Args:
        fixed_image
        moving_image
        
    Returns:
        registered_moving_image
    
    """
    
    #The the types of our images
    fixed_image_type = itk.Image[itk.D, 3]
    moving_image_type = itk.Image[itk.D, 3]
    output_image_type = itk.Image[itk.UC, 3]
    TransformType   = itk.TranslationTransform[itk.D, 3]
    
    #Instantiate the classes to do the registration
    
   
    transform    = TransformType.New()
    image_metric = itk.MutualInformationImageToImageMetric[fixed_image_type, moving_image_type].New()
    interpolator = itk.LinearInterpolateImageFunction[fixed_image_type, itk.D].New()
    optimizer    = itk.GradientDescentOptimizer.New()
    
    registration = itk.ImageRegistrationMethodv4.New(
    FixedImage = fixed_image,
    MovingImage=moving_image,
    Metric = image_metric,
    Optimizer = optimizer,
    InitialTransform = transform,
    )

    
    registration.SetFixedImage( fixed_image )
    registration.SetMovingImage( moving_image )
    
    registration.SetFixedImageRegion( fixed_image.GetLargestPossibleRegion() )
    
    transform.SetIdentity()
    initialParameters = transform.GetParameters()
    
    
    #Define the parameters for the optimizer
    """
    optimizer.SetScaleEstimator()
    optimizer.SetDoEstimateLearningRateAtEveryIteration( true )
    optimizer.SetDoEstimateLearningRateOnce( false )
    """
    optimizer.SetNumberOfIterations( 200 )
    
    
    #Start the registration
    registration.Update()
    
    print("Registration ended!")
    
    #Get the final parameters of the transform
    final_parameters = registration.GetLastTransformParameters()
    
    
    #use the final transform for resampling the moving image
    resampler = itk.ResampleImageFilter.New(
        Input=movingImage,
        Transform=outputCompositeTransform,
        UseReferenceImage=True,
        ReferenceImage=fixedImage,
    )
    resampler.SetDefaultPixelValue(100)
    
    output_cast = itk.RescaleIntensityImageFilter[fixed_image_type, output_image_type].New()
    output_cast.SetInput(resampler.GetOutput())
    
    return output_cast.Getoutput()